In [1]:
# !pip install -q xlrd
# !git clone https://github.com/skotak2/Seq2Seq-Machine-Translation-Model-Kannada-to-English.git



Cloning into 'Seq2Seq-Machine-Translation-Model-Kannada-to-English'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Receiving objects: 100% (92/92), 1.38 MiB | 550.00 KiB/s, done.
Resolving deltas: 100% (29/29), done.


In [2]:
%pip install torch


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 167.3/167.3 MB 1.3 MB/s eta 0:00:00
^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Importing the required libraries for building the nueral network
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#Defining Start and end of the sentense. SOS - start of the sentence and EOS - end of the sentences
SOS_token = 0
EOS_token = 1

#Creating word to index and frequency of each words in a dictionary. As the nueral network accepts input as a vector,
#We index every word with a number. Each unique number or vector could represent a word for input and output of text.
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [8]:
# Turn a Unicode string to plain ASCII

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [9]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines, 
    # the file in uploaded to the sessions cache of google colab, the file address needs to changed
    lines = open('data.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[s for s in l.split('\t')] for l in lines]
    l = []
    for p in pairs:
      l.append(p[0:2])
    pairs = l


    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [10]:
#Declaring function to format data into readable formats
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1,lang2,reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs




In [11]:
#The following is a sample list, [Kannada, English]
input_lang_kan, output_lang_eng, pairs = prepareData('eng','kan',True)
print(random.choice(pairs))

Reading lines...
Read 153 sentence pairs
Trimmed to 153 sentence pairs
Counting words...
Counted words:
kan 487
eng 496
['ಟಾಮ್ ಮೇರಿಗೆ ಮನೆಗೆ ಹೋಗುವುದಕ್ಕೆ ಹೇಳಿದನು.', 'Tom told Mary to go home.']


In [12]:
#Defining objects to create our lookup dictionaries for deployment
input_lkp = input_lang_kan.word2index
output_lkp = output_lang_eng.index2word

In [13]:
#We pickle the lookup libraries for deployment.
# picklefile = open('/content/drive/My Drive/Kan_Eng/input_kan', 'wb')
pickle.dump(input_lkp, open('input_kan', 'wb'))
# picklefile.close()

# picklefile = open('/content/drive/My Drive/Kan_Eng/output_eng', 'wb')
pickle.dump(output_lkp, open('output_eng', 'wb'))
# picklefile.close()

In [14]:
#Checking maximum length of sentence in the given dataset
for i in range(len(pairs)):
  a = 0
  b = 0
  if len(pairs[i][0].split()) > a:
    a = len(pairs[i][0].split())
  if len(pairs[i][1].split())>b:
    b = len(pairs[i][1].split())
print(a)
print(b)
#Maximum length of the kannada Sentence is 7
#maximum length of a english sentence is 14
#Hence Declaring Max_length as 15
MAX_LENGTH = 15

7
14


In [15]:
#Defining the class for the Encoder, the construct of layers and the RNN for encoding a given input sentence
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [16]:
#Decoder have the layers of nueral network and RNN as defined below, this class would work along with attention mechanism
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [17]:
#This class would use attention weights to each hidden state of encoder and build the context vector that would go in decoding the words in a sequence.
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
# We define the below functions to generate tensors from the text using the indexes assigned to each word. These tensors are used to feed the network
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang_kan, pair[0])
    target_tensor = tensorFromSentence(output_lang_eng, pair[1])
    return (input_tensor, target_tensor)

In [19]:
teacher_forcing_ratio = 0.5
# We define the training function below
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [20]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [21]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [22]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

ModuleNotFoundError: No module named 'matplotlib'

In [23]:
def evaluate_eng(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang_kan, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang_eng.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
            sen = ' '.join(decoded_words)

        return sen

In [24]:
#Training the model
hidden_size = 100
encoder_eng = EncoderRNN(input_lang_kan.n_words, hidden_size).to(device)
attn_decoder_eng = AttnDecoderRNN(hidden_size, output_lang_eng.n_words,dropout_p=0.1).to(device)

trainIters(encoder_eng, attn_decoder_eng, 75000, print_every=5000)

1m 39s (- 23m 19s) (5000 6%) 3.2155
3m 14s (- 21m 2s) (10000 13%) 0.3874
5m 1s (- 20m 5s) (15000 20%) 0.0467
6m 51s (- 18m 50s) (20000 26%) 0.0238
8m 43s (- 17m 26s) (25000 33%) 0.0160
10m 41s (- 16m 2s) (30000 40%) 0.0119
12m 33s (- 14m 21s) (35000 46%) 0.0095
14m 24s (- 12m 36s) (40000 53%) 0.0079
16m 11s (- 10m 47s) (45000 60%) 0.0067
17m 58s (- 8m 59s) (50000 66%) 0.0058
19m 55s (- 7m 14s) (55000 73%) 0.0052
21m 24s (- 5m 21s) (60000 80%) 0.0046


In [ ]:
#Save the weights for deployment
torch.save(encoder_eng.state_dict(), "model_enc_eng.dict")
torch.save(attn_decoder_eng.state_dict(), "model_dec_eng.dict")
print("complete")

complete


In [ ]:
#Evaluation for comparing the result
evaluate_eng(encoder_eng, attn_decoder_eng,"ಅವನು ನದಿ ಅಡ್ಡಲಾಗಿ ಈಜಿದನು.")

'He swam across the river.'

In [ ]:
txt = "ನಾವು ಹೊರಗೆ ಹೋಗಬಹುದಾ?"
try:
    evaluate_eng(encoder_eng, attn_decoder_eng,txt)
except KeyError:
    #get the word which is not in the dictionary
    p
    